In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from datetime import date, timedelta, datetime
from os.path import dirname, join, isfile, isdir
from bulletin import default_input, default_output
from bulletin.utils.normalize import normalize_cpf, normalize_cns, normalize_text, normalize_nome_mae, normalize_hash
from bulletin.systems import eSUS

path_file = join('C:\\', 'SESA', 'covid19datascience-dev', 'bulletin', 'database', 'gal')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
today = date.today()
someday = date(2020, 1, 2)
diff = today - someday

In [ ]:
# COLUNAS DO BANCO DO GAL

# Index(['id'0, 'origem_gal'1, 'requisicao'2, 'data_requisicao'3, 'data_solicitacao'4,
#         'requisicao_correlativo'5, 'unidade_requisitante'6,
#         'unidade_requisitante_municipio'7, 'unidade_requisitante_uf'8, 'paciente'9,
#         'cns'10, 'cpf'11, 'nome_mae'12, 'sexo'13, 'data_nascimento'14, 'idade'15,
#         'raca_cor'16, 'idade_gestacional'17, 'ibge'18, 'municipio'19, 'latitude'20,
#         'longitude'21, 'codigo_regional'22, 'regional'23, 'data_inicio_sintomas'24,
#         'cod_exame_requisicao'25, 'data_coleta'26, 'codigo_amostra'27,
#         'numero_amostra'28, 'numero_interno'29, 'material_biologico'30,
#         'data_encaminhamento_amostra'31, 'data_recebimento_amostra'32,
#         'tempo_transporte_amostra'33, 'envio_oportuno'34, 'exame'35, 'metodo'36,
#         'resultado'37, 'virus'38, 'data_liberacao'39, 'laboratorio_executor'40,
#         'laboratorio_executor_municipio'41, 'correlativo_exame'42,
#         'tempo_liberacao'43, 'tempo_diagnostico'44, 'codigo_status'45, 'status'46,
#         'classificacao'47],
#        dtype='object')]

In [ ]:
def foo(g,s):
    g = g.copy()
    s = s.copy()

    g.insert(0,'aaa',[ i for i in range(0,len(g))])

    g['cpf'] = g['cpf'].astype(str).str.split('.').apply(lambda x: x[0]).apply(normalize_cpf)
    g['cns'] = g['cns'].astype(str).str.split('.').apply(lambda x: x[0]).apply(normalize_cns)
    g['hmae'] = (g['paciente'].apply(normalize_text) + g['nome_mae'].apply(normalize_nome_mae)).apply(normalize_hash)

    s['NU_CPF'] = s['NU_CPF'].astype(str).str.split('.').apply(lambda x: x[0]).apply(normalize_cpf)
    s['NU_CNS'] = s['NU_CNS'].astype(str).str.split('.').apply(lambda x: x[0]).apply(normalize_cns)
    s['hmae'] = (s['NM_PACIENT'].apply(normalize_text) + s['NM_MAE_PAC'].apply(normalize_nome_mae)).apply(normalize_hash)

    gcpf = g.loc[g['cpf'].notna()].drop_duplicates('cpf',keep='last')
    gcns = g.loc[g['cns'].notna()].drop_duplicates('cns',keep='last')
    gmae = g.loc[g['hmae'].notna()].drop_duplicates('hmae',keep='last')

    scpf = s.loc[s['NU_CPF'].notna()].drop_duplicates('NU_CPF',keep='last')
    scns = s.loc[s['NU_CNS'].notna()].drop_duplicates('NU_CNS',keep='last')
    smae = s.loc[s['hmae'].notna()].drop_duplicates('hmae',keep='last')

    cpf = pd.merge(gcpf,scpf,how='inner',left_on='cpf',right_on='NU_CPF')
    cns = pd.merge(gcpf,scpf,how='inner',left_on='cns',right_on='NU_CNS')
    mae = pd.merge(gcpf,scpf,how='right',left_on='hmae',right_on='hmae')
    res = pd.concat([cpf,cns,mae], ignore_index=True).drop_duplicates('aaa',keep='first')
    
    
    #retirando os encontrados no GAL
#     res.set_index('requisicao', inplace=True)
#     g.set_index('requisicao', inplace=True)
#     g = g.drop(index=res.index)
#     res.reset_index(inplace=True)
#     g.reset_index(inplace=True)
    
    
    #retirnado os dados encontrados no SIVEP
    res.set_index('NU_NOTIFIC', inplace=True)
    s.set_index('NU_NOTIFIC', inplace=True)
    s = s.drop(index=res.index)
    res.reset_index(inplace=True)
    s.reset_index(inplace=True)
    
    
    res = res.drop_duplicates('NU_NOTIFIC',keep='first')
    
    
    print(len(res))
    res = res[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio','NU_NOTIFIC','NM_PACIENT','NU_CPF', 'NM_MAE_PAC', 'REQUI_GAL', 'DT_INTERNA']]
    return res, g, s

In [ ]:
## GAL 2020 - LACEN
gal_2020 = pd.read_pickle(join(path_file, 'banco_gal_2020-bz2.pkl'), compression='bz2')
gal = pd.concat([gal_2020])
projeto = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
projeto = projeto.loc[projeto['resultado'] == 'Detectável']
projeto = projeto.loc[projeto['laboratorio_executor'] == 'Laboratório Central do Estado']

projeto = projeto.loc[projeto['idade'].str.endswith("Ano(s)")].copy()
projeto['idade'] = projeto['idade'].str[:2]
projeto['idade'] = projeto['idade'].astype('int')

grupo_1 = projeto.loc[(projeto['idade'] >= 18) & (projeto['idade'] <= 43)]
grupo_1.to_pickle(join(default_output, 'projeto','GAL-grupo01_2020.pkl'))
print(len(grupo_1))

grupo_2 = projeto.loc[(projeto['idade'] >= 75)]
grupo_2.to_pickle(join(default_output, 'projeto','GAL-grupo02_2020.pkl'))
print(len(grupo_2))



## GAL 2020 - OUTROS LABS
projeto = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
projeto = projeto.loc[projeto['resultado'] == 'Detectável']
projeto = projeto.loc[projeto['laboratorio_executor'] != 'Laboratório Central do Estado']

projeto = projeto.loc[projeto['idade'].str.endswith("Ano(s)")].copy()
projeto['idade'] = projeto['idade'].str[:2]
projeto['idade'] = projeto['idade'].astype('int')


grupo_1 = projeto.loc[(projeto['idade'] >= 18) & (projeto['idade'] <= 43)]
grupo_1.to_pickle(join(default_output, 'projeto','GAL-grupo01_2020-lab.pkl'))
print(len(grupo_1))
                      
grupo_2 = projeto.loc[(projeto['idade'] >= 75)]
grupo_2.to_pickle(join(default_output, 'projeto','GAL-grupo02_2020-lab.pkl'))
print(len(grupo_2))

In [ ]:
## GAL 2021 - LACEN
gal_2021 = pd.read_pickle(join(path_file, 'banco_gal_2021-bz2.pkl'), compression='bz2')
gal_2021_complemento = pd.read_csv(join(path_file, 'consulta-gal-complemento-2021.csv'), sep = ';')
gal = pd.concat([gal_2021, gal_2021_complemento])


projeto = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
projeto = projeto.loc[projeto['resultado'] == 'Detectável']
projeto = projeto.loc[projeto['laboratorio_executor'] == 'Laboratório Central do Estado']

projeto = projeto.loc[projeto['idade'].str.endswith("Ano(s)")].copy()
projeto['idade'] = projeto['idade'].str[:2]
projeto['idade'] = projeto['idade'].astype('int')

grupo_1 = projeto.loc[(projeto['idade'] >= 18) & (projeto['idade'] <= 43)]
grupo_1.to_pickle(join(default_output, 'projeto','GAL-grupo01_2021.pkl'))
print(len(grupo_1))

grupo_2 = projeto.loc[(projeto['idade'] >= 75)]
grupo_2.to_pickle(join(default_output, 'projeto','GAL-grupo02_2021.pkl'))
print(len(grupo_2))



## GAL 2021 - OUTROS LABS
projeto = gal.loc[gal["virus"] == 'Coronavírus SARS-CoV2']
projeto = projeto.loc[projeto['resultado'] == 'Detectável']
projeto = projeto.loc[projeto['laboratorio_executor'] != 'Laboratório Central do Estado']

projeto = projeto.loc[projeto['idade'].str.endswith("Ano(s)")].copy()
projeto['idade'] = projeto['idade'].str[:2]
projeto['idade'] = projeto['idade'].astype('int')


grupo_1 = projeto.loc[(projeto['idade'] >= 18) & (projeto['idade'] <= 43)]
grupo_1.to_pickle(join(default_output, 'projeto','GAL-grupo01_2021-lab.pkl'))
print(len(grupo_1))
                      
grupo_2 = projeto.loc[(projeto['idade'] >= 75)]
grupo_2.to_pickle(join(default_output, 'projeto','GAL-grupo02_2021-lab.pkl'))
print(len(grupo_2))

In [ ]:
g20201 = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo01_2020.pkl'))
g20201 = g20201.sort_values('data_liberacao')
print(len(g20201))
g20202 = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo02_2020.pkl'))
g20202 = g20202.sort_values('data_liberacao')
print(len(g20202))

In [ ]:
g20211 = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo01_2021.pkl'))
g20211 = g20211.sort_values('data_liberacao')
print(len(g20211))
g20212 = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo02_2021.pkl'))
g20212 = g20212.sort_values('data_liberacao')
print(len(g20212))

## 2020

### 2020 G1

In [ ]:
s20201 = pd.read_pickle(join(default_output,'projeto', 'sivep_2020_g1.pkl'))
s20201 = s20201.sort_values('DT_INTERNA')
s20201_r = s20201.loc[~s20201['REQUI_GAL'].isna()]
s20201 = s20201.loc[s20201['REQUI_GAL'].isna()]
s20201_r['REQUI_GAL'] = s20201_r['REQUI_GAL'].astype('int64')
s20201_r = s20201_r.drop_duplicates('REQUI_GAL', keep='first')
print(len(s20201_r) + len(s20201))

In [ ]:
#G1 - VERIFICAR QUAL ENCONTROU e QUAL NÃO ENCONTROU
founded_20201 = g20201.loc[g20201['requisicao'].isin(s20201_r['REQUI_GAL'])] #arquivo final parcial
print(len(founded_20201))

#G1 - RETIRAR DO BANCO DO SIVEP E GAL OS QUE ENCONTRARAM
g20201.set_index('requisicao', inplace=True)
founded_20201.set_index('requisicao', inplace=True)
s20201_r.set_index('REQUI_GAL', inplace=True)

g20201 = g20201.drop(index=founded_20201.index)
s20201_r = s20201_r.drop(index=founded_20201.index)

g20201.reset_index(inplace=True)
founded_20201.reset_index(inplace=True)
s20201_r.reset_index(inplace=True)

s20201 = pd.concat([s20201, s20201_r])

#G1 - FILTRAR AS COLUNAS DO ARQUIVO GAL PARCIAL
founded_20201 = founded_20201[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

In [ ]:
res_g20201_s20201, g20201, s20201 = foo(g20201,s20201)

In [ ]:
g20201_t = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo01_2020-lab.pkl'))
print(len(g20201_t))

res_g20201_s20201_more_labs, g20201, s20201 = foo(g20201_t,s20201)

In [ ]:
res_final_g20201_s20201 = pd.concat([res_g20201_s20201, res_g20201_s20201_more_labs])

res_final_g20201_s20201 = res_final_g20201_s20201.loc[ res_final_g20201_s20201['paciente'] == res_final_g20201_s20201['NM_PACIENT'] ]

res_final_g20201_s20201 = res_final_g20201_s20201[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

res_final_g20201_s20201 = pd.concat([res_final_g20201_s20201, founded_20201])

res_final_g20201_s20201.to_csv(join(default_output,'projeto','LACEN_G01_2020.csv'), index=False, sep=';', encoding='utf-8-sig')

print(len(res_final_g20201_s20201))
table = res_final_g20201_s20201.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_1.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

### 2020 G2

In [ ]:
s20202 = pd.read_pickle(join(default_output,'projeto', 'sivep_2020_g2.pkl'))
s20202 = s20202.sort_values('DT_INTERNA')
s20202_r = s20202.loc[~s20202['REQUI_GAL'].isna()]
s20202 = s20201.loc[s20201['REQUI_GAL'].isna()]
s20202_r['REQUI_GAL'] = s20202_r['REQUI_GAL'].astype('int64')
s20202_r = s20202_r.drop_duplicates('REQUI_GAL', keep='first')
print(len(s20202_r) + len(s20202))

In [ ]:
#G1 - VERIFICAR QUAL ENCONTROU e QUAL NÃO ENCONTROU
founded_20202 = g20202.loc[g20202['requisicao'].isin(s20202_r['REQUI_GAL'])] #arquivo final parcial
print(len(founded_20202))

#G1 - RETIRAR DO BANCO DO SIVEP E GAL OS QUE ENCONTRARAM
g20202.set_index('requisicao', inplace=True)
founded_20202.set_index('requisicao', inplace=True)
s20202_r.set_index('REQUI_GAL', inplace=True)

g20202 = g20202.drop(index=founded_20202.index)
s20202_r = s20202_r.drop(index=founded_20202.index)

g20202.reset_index(inplace=True)
founded_20202.reset_index(inplace=True)
s20202_r.reset_index(inplace=True)

s20202 = pd.concat([s20202, s20202_r])

#G1 - FILTRAR AS COLUNAS DO ARQUIVO GAL PARCIAL
founded_20202 = founded_20202[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

In [ ]:
res_g20202_s20202, g20202, s20202 = foo(g20202,s20202)

In [ ]:
g20202_t = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo02_2020-lab.pkl'))
print(len(g20202_t))

res_g20202_s20202_more_labs, g20202, s20202 = foo(g20202_t,s20202)

In [ ]:
res_final_g20202_s20202 = pd.concat([res_g20202_s20202, res_g20202_s20202_more_labs])

res_final_g20202_s20202 = res_final_g20202_s20202.loc[ res_final_g20202_s20202['paciente'] == res_final_g20202_s20202['NM_PACIENT'] ]

res_final_g20202_s20202 = res_final_g20202_s20202[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

res_final_g20202_s20202 = pd.concat([res_final_g20202_s20202, founded_20202])

res_final_g20202_s20202.to_csv(join(default_output,'projeto','LACEN_G02_2020.csv'), index=False, sep=';', encoding='utf-8-sig')


# res_final_g20202_s20202 = res_final_g20202_s20202.drop_duplicates('cpf',keep='first')
print(len(res_final_g20202_s20202))
table = res_final_g20202_s20202.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_2.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

## 2021

### 2021 G1

In [ ]:
s20211 = pd.read_pickle(join(default_output,'projeto', 'sivep_2021_g1.pkl'))
s20211 = s20211.sort_values('DT_INTERNA')
s20211_r = s20211.loc[~s20211['REQUI_GAL'].isna()]
s20211 = s20211.loc[s20211['REQUI_GAL'].isna()]
s20211_r['REQUI_GAL'] = s20211_r['REQUI_GAL'].astype('int64')
s20211_r = s20211_r.drop_duplicates('REQUI_GAL', keep='first')
print(len(s20211_r) + len(s20211))

In [ ]:
s20211['CO_MUN_RES'].unique()

In [ ]:
list_ibge = [314340, 500570, 210370, 330455, 430750, 351340]
s20211.loc[s20211['SG_UF'] != 'PR']

In [ ]:
#G1 - VERIFICAR QUAL ENCONTROU e QUAL NÃO ENCONTROU
founded_20211 = g20211.loc[g20211['requisicao'].isin(s20211_r['REQUI_GAL'])] #arquivo final parcial
print(len(founded_20211))

#G1 - RETIRAR DO BANCO DO SIVEP E GAL OS QUE ENCONTRARAM
g20211.set_index('requisicao', inplace=True)
founded_20211.set_index('requisicao', inplace=True)
s20211_r.set_index('REQUI_GAL', inplace=True)

g20211 = g20211.drop(index=founded_20211.index)
s20211_r = s20211_r.drop(index=founded_20211.index)

g20211.reset_index(inplace=True)
founded_20211.reset_index(inplace=True)
s20211_r.reset_index(inplace=True)

s20211 = pd.concat([s20211, s20211_r])

#G1 - FILTRAR AS COLUNAS DO ARQUIVO GAL PARCIAL
founded_20211 = founded_20211[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

In [ ]:
res_g20211_s20211, g20211, s20211 = foo(g20211,s20211)

In [ ]:
g20211_t = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo01_2021-lab.pkl'))
g20211_t = g20211_t.sort_values('data_liberacao')
g20211_t = g20211_t.loc[g20211_t['laboratorio_executor'] != 'Instituto de Biologia Molecular do Paraná - IBMP']

print(len(g20211_t))

res_g20211_s20211_more_labs, g20211, s20211 = foo(g20211_t,s20211)

In [ ]:
res_final_g20211_s20211 = pd.concat([res_g20211_s20211, res_g20211_s20211_more_labs])

res_final_g20211_s20211 = res_final_g20211_s20211.loc[ res_final_g20211_s20211['paciente'] == res_final_g20211_s20211['NM_PACIENT'] ]

res_final_g20211_s20211 = res_final_g20211_s20211[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

res_final_g20211_s20211 = pd.concat([res_final_g20211_s20211, founded_20211])

res_final_g20211_s20211.to_csv(join(default_output,'projeto','LACEN_G01_2021.csv'), index=False, sep=';', encoding='utf-8-sig')

print(len(res_final_g20211_s20211))
table = res_final_g20211_s20211.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_3.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

### 2021 G2

In [ ]:
s20212 = pd.read_pickle(join(default_output,'projeto', 'sivep_2021_g2.pkl'))
s20212 = s20212.sort_values('DT_INTERNA')
s20212_r = s20212.loc[~s20212['REQUI_GAL'].isna()]
s20212 = s20212.loc[s20212['REQUI_GAL'].isna()]
s20212_r['REQUI_GAL'] = s20212_r['REQUI_GAL'].astype('int64')
s20212_r = s20212_r.drop_duplicates('REQUI_GAL', keep='first')
print(len(s20212_r) + len(s20212))

In [ ]:
s20212['CO_MUN_RES'].unique()

In [ ]:
#G2 - VERIFICAR QUAL ENCONTROU e QUAL NÃO ENCONTROU
founded_20212 = g20212.loc[g20212['requisicao'].isin(s20212_r['REQUI_GAL'])] #arquivo final parcial
print(len(founded_20212))

#G2 - RETIRAR DO BANCO DO SIVEP E GAL OS QUE ENCONTRARAM
g20212.set_index('requisicao', inplace=True)
founded_20212.set_index('requisicao', inplace=True)
s20212_r.set_index('REQUI_GAL', inplace=True)

g20212 = g20212.drop(index=founded_20212.index)
s20212_r = s20212_r.drop(index=founded_20212.index)

g20212.reset_index(inplace=True)
founded_20212.reset_index(inplace=True)
s20212_r.reset_index(inplace=True)

s20212 = pd.concat([s20212, s20212_r])

#G2 - FILTRAR AS COLUNAS DO ARQUIVO GAL PARCIAL
founded_20212 = founded_20212[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

In [ ]:
res_g20212_s20212, g20212, s20212 = foo(g20212,s20212)

In [ ]:
g20212_t = pd.read_pickle(join(default_output,'projeto', 'GAL-grupo02_2021-lab.pkl'))
g20212_t = g20212_t.sort_values('data_liberacao')
g20212_t = g20212_t.loc[g20212_t['laboratorio_executor'] != 'Instituto de Biologia Molecular do Paraná - IBMP']

print(len(g20212_t))

res_g20212_s20212_more_labs, g20212, s20212 = foo(g20212_t,s20212)

In [ ]:
res_final_g20212_s20212 = pd.concat([res_g20212_s20212, res_g20212_s20212_more_labs])

res_final_g20212_s20212 = res_final_g20212_s20212.loc[ res_final_g20212_s20212['paciente'] == res_final_g20212_s20212['NM_PACIENT'] ]

res_final_g20212_s20212 = res_final_g20212_s20212[['requisicao', 'data_requisicao', 'paciente', 'cpf', 'cns', 'nome_mae', 'municipio', 'data_coleta', 'codigo_amostra', 'numero_amostra', 'numero_interno', 'material_biologico', 'exame', 'metodo', 'resultado', 'virus', 'data_liberacao', 'laboratorio_executor', 'laboratorio_executor_municipio']]

res_final_g20212_s20212 = pd.concat([res_final_g20212_s20212, founded_20212])

res_final_g20212_s20212.to_csv(join(default_output,'projeto','LACEN_G02_2021.csv'), index=False, sep=';', encoding='utf-8-sig')

print(len(res_final_g20212_s20212))
table = res_final_g20212_s20212.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_4.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

## VERIFICAÇÃO DA SITUAÇÃO VACINAL

In [ ]:
tipos = {
    'cns':str,
    'cpf':str
}

In [ ]:
LACEN_G01_2021 = pd.read_csv(join(default_output,'projeto','LACEN_G01_2021.csv'), sep=';', encoding='utf-8-sig', dtype=tipos, parse_dates=['data_coleta'])
LACEN_G01_2021['g'] = 1
LACEN_G02_2021 = pd.read_csv(join(default_output,'projeto','LACEN_G02_2021.csv'), sep=';', encoding='utf-8-sig', dtype=tipos, parse_dates=['data_coleta'])
LACEN_G02_2021['g'] = 2
lacen = pd.concat([LACEN_G01_2021,LACEN_G02_2021]).reset_index().rename(columns={'index':'id'})

In [ ]:
vac = eSUS()
vac.load('vacinados')

In [ ]:
lacen['hash_mae'] = (lacen['paciente'].apply(normalize_text) + lacen['nome_mae'].apply(normalize_nome_mae)).apply(normalize_hash)

In [ ]:
vacinados = vac.df

casos_vacinados_cns = pd.merge(vacinados,lacen[['cns','id','data_coleta']], on='cns', how='inner')
print(len(casos_vacinados_cns))

casos_vacinados_cpf = pd.merge(vacinados.loc[vacinados['cpf'].notna()],lacen[['cpf','id','data_coleta']].rename(columns={'NU_CPF':'cpf'}), on='cpf', how='inner')
print(len(casos_vacinados_cpf))

casos_vacinados_mae = pd.merge(vacinados.loc[vacinados['hash_mae'].notna()],lacen[['hash_mae','id','data_coleta']], on='hash_mae', how='inner')
print(len(casos_vacinados_mae))

casos_vacinados = pd.concat([casos_vacinados_cns,casos_vacinados_cpf,casos_vacinados_mae]).drop_duplicates('id', keep='first')
print(len(casos_vacinados))

In [ ]:
casos_vacinados = casos_vacinados.rename(columns={'data_coleta':'data_diagnostico'})

casos_vacinados['dias_entre_dose_diagnostico'] = 0
'''
se 
    a data de diagnóstico for anterior a vacinação
    ou inconsistencia
entao -1
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'inconsistencia') |
    (casos_vacinados['data_aplicacao_1a_dose'] > casos_vacinados['data_diagnostico'])
), 'dias_entre_dose_diagnostico'] = -1

'''
se 
    só a primeira dose foi aplicada
então    
    dias_entre_dose_diagnostico será data_diagnostico - data_aplicacao_1a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'só primeira dose') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico'])
), 'data_aplicacao_1a_dose']).dt.days

'''
se 
    ambas as doses foram aplicadas
    e a data de diagnóstico está entre as doses 
então    
    dias_entre_dose_diagnostico será data_diagnostico - data_aplicacao_1a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_1a_dose'] <= casos_vacinados['data_diagnostico']) &
    (casos_vacinados['data_aplicacao_2a_dose'] >= casos_vacinados['data_diagnostico'])
),'data_aplicacao_1a_dose']).dt.days

'''
se 
    ambas as doses foram aplicadas
    e a data de diagnóstico está depois da 2 dose
então    
    os dias entre dose e diagnóstico será data_diagnostico - data_aplicacao_2a_dose
'''
casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'dias_entre_dose_diagnostico'] = (casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'data_diagnostico'] - casos_vacinados.loc[(
    (casos_vacinados['situacao'] == 'ambas doses aplicadas') &
    (casos_vacinados['data_aplicacao_2a_dose'] <= casos_vacinados['data_diagnostico'])
),'data_aplicacao_2a_dose']).dt.days

casos_vacinados['vacinado'] = 0

casos_vacinados.loc[
    (
        (casos_vacinados['situacao']=='ambas doses aplicadas') & 
        (casos_vacinados['data_diagnostico'] <= casos_vacinados['data_aplicacao_2a_dose'] ) &
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'] )
    ) | (
        (casos_vacinados['situacao']=='só primeira dose') & 
        (casos_vacinados['data_diagnostico'] >= casos_vacinados['data_aplicacao_1a_dose'] )
    )
,'vacinado'] = 1 # contaminação entre as doses | vacinado com uma dose porém não imunizado

casos_vacinados.loc[
    (casos_vacinados['situacao']=='dose unica aplicada') & 
    (casos_vacinados['data_diagnostico'] > (casos_vacinados['data_aplicacao_2a_dose']) )
,'vacinado'] = 2 # contaminação depois da dose unica | vacinados com a dose unica porém não imunizado

casos_vacinados.loc[
    (casos_vacinados['situacao']=='ambas doses aplicadas') & 
    (casos_vacinados['data_diagnostico'] > (casos_vacinados['data_aplicacao_2a_dose']) )
,'vacinado'] = 3 # contaminação depois da segunda dose | vacinados com as duas dose porém não imunizado

casos_vacinados.loc[
    (casos_vacinados['situacao']=='dose unica aplicada') & 
    (casos_vacinados['data_diagnostico'] > (casos_vacinados['data_aplicacao_2a_dose'] + timedelta(14)) )
,'vacinado'] = 4 # contaminação depois da imunização dose unica 

casos_vacinados.loc[
    (casos_vacinados['situacao']=='ambas doses aplicadas') & 
    (casos_vacinados['data_diagnostico'] > (casos_vacinados['data_aplicacao_2a_dose'] + timedelta(14)) )
,'vacinado'] = 5 # contaminação depois da imunização com as duas dose

In [ ]:
casos_vacinados

In [ ]:
casos = pd.merge(lacen,casos_vacinados[['id','vacinado']],how='left',on='id')
casos['vacinado'] = casos['vacinado'].fillna(0)
casos = casos.replace({'vacinado':{0: 'Não vacinado', 1: 'Entre as doses', 2: 'Após dose única', 3:'Após 2ª dose', 4: 'Após imunizado dose única', 5: 'Após imunizado 2ª dose'}})

In [ ]:
casos.groupby(['vacinado','g'])[['id']].count().unstack(-1)

In [ ]:
LACEN_G01_2021_NAO_VACINADO = casos.loc[(casos['g']==1)&(casos['vacinado']=='Não vacinado')].drop(columns=['g','vacinado','hash_mae'])
LACEN_G02_2021_NAO_VACINADO = casos.loc[(casos['g']==2)&(casos['vacinado']=='Não vacinado')].drop(columns=['g','vacinado','hash_mae'])

In [ ]:
table = LACEN_G01_2021_NAO_VACINADO.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_5.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

In [ ]:
table = LACEN_G02_2021_NAO_VACINADO.groupby('laboratorio_executor')[['paciente']].count().reset_index()
table.to_csv(join(default_output,'projeto', 'table_6.csv'), index=False, sep=';', encoding='utf-8-sig')
display(table)

In [ ]:
LACEN_G01_2021_NAO_VACINADO.to_csv(join(default_output,'projeto','LACEN_G01_2021_NAO_VACINADO.csv'), index=False, sep=';', encoding='utf-8-sig')
LACEN_G02_2021_NAO_VACINADO.to_csv(join(default_output,'projeto','LACEN_G02_2021_NAO_VACINADO.csv'), index=False, sep=';', encoding='utf-8-sig')

In [ ]:
old = pd.read_excel('LACEN_G02_2020.xlsx')
print(len(old))
list_old = set(old['requisicao'])

In [ ]:
new = pd.read_excel('LACEN_G02_2020_new.xlsx')
print(len(new))
list_new = set(new['requisicao'])

In [ ]:
teste = pd.concat([old, new])
teste = teste.drop_duplicates('requisicao', keep='first')
testao = set(teste['requisicao'])

In [ ]:
conjunto_resultante = list_new.difference(list_old)
print(len(conjunto_resultante))
conjunto_resultante

In [ ]:
search=200206006835.0

In [ ]:
old.loc[old['requisicao']==search]

In [ ]:
new.loc[new['requisicao']==search]

In [ ]:
resultado = new.loc[new['requisicao'].isin(conjunto_resultante)]

In [ ]:
resultado.to_excel('LACEN_grupo02.xlsx', index=False)